In [1]:
import csv
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers

RANDOM_SEED = 42

In [2]:
dataset_test = 'glove_data_test/glove_test_data.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier_glove.keras'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier_glove.tflite'

In [3]:
NUM_CLASSES = 24

In [4]:
# Directory containing CSV files
directory = 'data_glove'

# Initialize an empty list to store data from all files
data_X = []
data_y = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        # Load data from each file and append to the list
        data_X.append(np.loadtxt(file_path, delimiter=',', dtype='float32', usecols=list(range(300, 325)) + list(range(4, 151))))
        data_y.append(np.loadtxt(file_path, delimiter=',', dtype='float32', usecols=(0)))

# Concatenate data from all files along the first axis
X_dataset = np.concatenate(data_X, axis=0)
y_dataset = np.concatenate(data_y, axis=0)

X_train, X_validation, y_train, y_validation = train_test_split(X_dataset, y_dataset, train_size=0.70, random_state=RANDOM_SEED)
X_test = np.loadtxt(dataset_test, delimiter=',', dtype='float32', usecols=list(range(300, 325)) + list(range(4, 151)))
y_test = np.loadtxt(dataset_test, delimiter=',', dtype='int32', usecols=(0))

In [14]:
# Initialize empty lists for specialised GQ training
X_train_GQ= []
Y_train_GQ=[]


for i, int_value in enumerate(y_train):
    if int_value == 6 or int_value ==15:
        X_train_GQ.append(X_train[i])
        Y_train_GQ.append(y_train[i])

# Initialize empty lists for specialised GQ testing
X_test_GQ= []
Y_test_GQ=[]


for i, int_value in enumerate(y_test):
    if int_value == 6 or int_value ==15:
        X_test_GQ.append(X_test[i])
        Y_test_GQ.append(y_test[i])

X_validation_GQ=[]
Y_validation_GQ=[]
for i, int_value in enumerate(y_validation):
    if int_value == 6 or int_value ==15:
        X_validation_GQ.append(X_validation[i])
        Y_validation_GQ.append(y_validation[i])

print(Y_train_GQ.shape)
print(X_validation_GQ.shape)
print(Y_validation_GQ.shape)
print(X_test_GQ.shape)
print(Y_test_GQ.Shape)

AttributeError: 'list' object has no attribute 'shape'

In [10]:
GQ_CLASSES = 2
GQmodel = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((172, 1), input_shape=(172,)),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=10),
    tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(GQ_CLASSES, activation='softmax')
])

GQmodel.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1)

history = GQmodel.fit(
    X_train_GQ,
    Y_train_GQ,
    epochs=1000,
    batch_size=128,
    validation_data=(X_validation_GQ, Y_validation_GQ),
    callbacks=[cp_callback, es_callback]
)

val_loss, val_acc = GQmodel.evaluate(X_test_GQ, Y_test_GQ, batch_size = 4)

IndexError: tuple index out of range